# Data Cleaning

Source of the dataset: http://insideairbnb.com/get-the-data.html

I will be using the detailled dataset of listings because it has many interesting features. 
- Shape of listings csv is: (65493, 16)
- Shape of listings_detailed csv is: (65493, 106)

I will use the data scrapped last November (November 7, 2019) because the data won't depend on a crisis context.

### Required actions:
- drop duplicates and useless columns (with too many text description for example)
- check and fix missing values
- check data types and fix columns (43,61,62) that have mixed types
- add new calculated columns to simplify the comprehension of data
- transform f/t categorical columns to numerical 0/1
- check other categorical columns and reduce or transform into numerical data if possible
- transform dummies (need to think again about depending on the model I will use)


categorical columns to be reduced: property_type, city, guests_included

calculated columns: time_since_last_review, time_since_host

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

%matplotlib inline
sns.set()

pd.set_option('max_columns',106)

In [ ]:
listing_full = pd.read_csv('http://data.insideairbnb.com/france/ile-de-france/paris/2019-11-07/data/listings.csv')
print(listing_full.shape)
listing_full.head()

____________________________________
### Drop duplicates

Checking duplicates and dropping them just in case

In [ ]:
listing_full.shape

In [ ]:
listing_full = listing_full.drop_duplicates()
print(listing_full.shape)

____________________________________
### Drop useless columns

In [ ]:
df = listing_full.copy()
col_drop = []

In [ ]:
col_drop.extend(listing_full.iloc[:,:21].columns)
col_drop.extend(['host_name','host_about','host_thumbnail_url', 'host_has_profile_pic',
                 'host_picture_url','host_neighbourhood','host_listings_count','host_verifications',
                 'street','neighbourhood','neighbourhood_group_cleansed','state','zipcode','market',
                 'smart_location','country_code','country','latitude','longitude','amenities',
                 'calendar_updated','calendar_last_scraped','first_review','requires_license',
                 'license','jurisdiction_names','has_availability'])

In [ ]:
col_drop = list(set(col_drop))
col_drop

In [ ]:
# Checking the columns and their values 20 by 20

df.iloc[:,80:].head()

In [ ]:
# Checking the different values for one column

df[['calculated_host_listings_count','host_total_listings_count']]

In [ ]:
# Dropping the columns

df = df.drop(columns=col_drop)
print(df.shape)

_________________________
### Checking correlation to drop correlated columns and avoid multicollinearity

In [ ]:
df1 = df.copy()
col_drop = []
df1.columns

In [ ]:
col_drop.extend(['calculated_host_listings_count_entire_homes','availability_30',
                 'availability_60','availability_90','minimum_minimum_nights','maximum_minimum_nights',
                 'minimum_maximum_nights','maximum_maximum_nights','minimum_nights_avg_ntm',
                 'maximum_nights_avg_ntm','calculated_host_listings_count','beds','number_of_reviews_ltm'])

In [ ]:
# Dropping the columns

df1 = df1.drop(columns=col_drop)
print(df1.shape)

In [ ]:
# Checking correlation between columns 

plt.figure(figsize=(17,10))
sns.heatmap(df1.corr());

______________________
### Checking and fixing missing values

In [ ]:
df2 = df1.copy()
col_drop = []
row_drop = []

null_col = df2.isna().sum()
round(null_col[null_col>0]/df2.shape[0]*100,2)

In [ ]:
# Dropping all the review columns because it's not the most important feature for pricing and
# there are too much missing values to handle

# neighbourhood_cleansed seems to be really accurate so we can drop city which seems not so accurate

col_drop.extend(['city','square_feet','weekly_price','monthly_price','host_location','host_acceptance_rate',
                 'security_deposit','cleaning_fee','review_scores_rating','review_scores_accuracy',
                 'review_scores_cleanliness','review_scores_checkin','review_scores_communication',
                 'review_scores_location','review_scores_value','reviews_per_month','last_review'])

row_drop.extend(df2[df2.host_since.isna()].index)

In [ ]:
# Dropping columns and rows

df2 = df2.drop(columns=col_drop)
df2 = df2.drop(index=row_drop)
print(df2.shape)

In [ ]:
df2.host_response_time = df2.host_response_time.fillna('None')

In [ ]:
# Handle nan values of host_response_rate. Nan values may mean the host has never received any message to answer.

# Filling nan values and rid off % to convert values into integer and allow creation of bins
df2.host_response_rate = df2.host_response_rate.fillna('-1').apply(lambda x: x.strip('%')).astype(int)

# Creating bins for response rate
df2.host_response_rate = pd.cut(df2.host_response_rate,[-2,-0.001,25,50,75,100], 
                                labels=['None','0-25%','25-50%','50-75%','75-100%'])

In [ ]:
# Filling missing vallues for bathrooms and bedrooms by 1.0
# After checking the most frequent value is 1.0 for any accommodates (2,3,4,5) and any room_type

print("bathrooms mode:",df2[(df2.accommodates==2)&(df2.room_type=='Entire home/apt')].bathrooms.mode())
df2.bathrooms = df2.bathrooms.fillna(1.0)

print("bedrooms mode",df2[(df2.accommodates==2)&(df2.room_type=='Entire home/apt')].bedrooms.mode())
df2.bedrooms = df2.bedrooms.fillna(1.0)

In [ ]:
null_col = df2.isna().sum()
round(null_col[null_col>0]/df2.shape[0]*100,2)

_______________________________
### Cleaning dtypes

In [ ]:
df3 = df2.copy()
df3.dtypes

In [ ]:
df3[['price','extra_people']] = df3[['price','extra_people']].applymap(lambda x: x.strip('$').replace(',','')).astype(float)



In [ ]:
df3.host_since = pd.to_datetime(df3.host_since)

In [ ]:
df3.dtypes

________________________________
### Creating new columns

In [ ]:
from datetime import datetime

current_date = datetime.today()
df3['time_since_host'] = (current_date - df3.host_since).dt.days
df3 = df3.drop('host_since',axis=1)
df3.head()

______________________
### Transform categorical data

In [ ]:
df4 = df3.copy()

In [ ]:
# Reducing the list of property_type by keeping top 10 and put other categories into Other

top_10_prop = df4.property_type.value_counts().head(10).index
df4.property_type = df4.property_type.apply(lambda x: x if x in top_10_prop else 'Other')
df4.property_type.value_counts()

In [ ]:
# Transforming boolean values into integer

df4[['host_is_superhost','host_identity_verified',
     'is_location_exact','instant_bookable',
     'is_business_travel_ready','require_guest_profile_picture',
     'require_guest_phone_verification']] = df4[['host_is_superhost','host_identity_verified',
     'is_location_exact','instant_bookable',
     'is_business_travel_ready','require_guest_profile_picture',
     'require_guest_phone_verification']].applymap(lambda x: 1 if x=='t' else 0)

In [ ]:
df4.shape

In [ ]:
# Saving the csv without dummies for EDA

df4.to_csv('../data/airbnb_paris_clean.csv',index=False)

In [ ]:
df5 = df4.copy()
df5 = pd.get_dummies(df5,drop_first=True)

In [ ]:
df5.shape

In [ ]:
# Saving the csv with dummies

df5.to_csv('../data/airbnb_paris_clean_dummies.csv',index=False)